<a href="https://colab.research.google.com/github/nug1209/PwC_Switzerland_Digital_Intelligence_Virtual_Case_Experience/blob/main/PwC_Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this task we deal with merchant loan data. We will create a forecast about how much will the total repayment will be after these loans are finished (after 30 months since the loans started). The formula for counting the repayment is provided. We will also create percentage values for each repayment in comparison with the origination amount.

In [217]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from datetime import datetime

In [218]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Virtual Experience/PwC_Task_2_Data.csv'

df = pd.read_csv(path, delimiter=';')
df_copy = df.copy()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [219]:
# df

In [220]:
df = df.rename(columns={'Unnamed: 0':'origination_date', 'Origination Amount':'origination_amount'})
# df.head(3)

In [221]:
df.shape

(20, 22)

In [222]:
to_remove = ['origination_date', 'origination_amount']
value_columns = [i for i in list(df.columns) if i not in to_remove]

df = pd.melt(df, id_vars=['origination_date', 'origination_amount'], value_vars=value_columns)
df = df.rename(columns={'variable':'repayment_date', 'value':'repayment_amount'})

In [223]:
df = df[df['repayment_amount'] != 0.00]
# df

In [224]:
df['origination_date'] = pd.to_datetime(df['origination_date'], format='%d.%m.%Y')
df['repayment_date'] = pd.to_datetime(df['repayment_date'], format='%d.%m.%Y')
# df.info()

In [225]:
df = df.sort_values(by=['origination_date', 'repayment_date'])
df

,origination_date,origination_amount,repayment_date,repayment_amount
0,2019-05-31,10018746.17,2019-05-31,1443069.08
20,2019-05-31,10018746.17,2019-06-30,3332200.33
40,2019-05-31,10018746.17,2019-07-31,1328138.75
60,2019-05-31,10018746.17,2019-08-31,928085.74
80,2019-05-31,10018746.17,2019-09-30,736418.27
...,...,...,...,...
377,2020-10-31,27699586.46,2020-11-30,8752706.73
397,2020-10-31,27699586.46,2020-12-31,1503544.68
378,2020-11-30,29872889.68,2020-11-30,4383982.78
398,2020-11-30,29872889.68,2020-12-31,8383025.07


In [226]:
# for i in value_columns:
#   df[i + '_actual_repayment_percent'] = 100 * df[i] / df['origination_amount']

# df.columns

In [227]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210 entries, 0 to 399
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   origination_date    210 non-null    datetime64[ns]
 1   origination_amount  210 non-null    float64       
 2   repayment_date      210 non-null    datetime64[ns]
 3   repayment_amount    210 non-null    float64       
dtypes: datetime64[ns](2), float64(2)
memory usage: 8.2 KB


In [233]:
grouped = df.groupby('origination_date')['repayment_date']

## Get the second repayment (p2)

df[df['repayment_date'] == grouped.transform(lambda x: x.nsmallest(2).max())][['origination_date', 'repayment_amount']].rename(columns={'repayment_amount':'p2'})

# df['p2'] = df[df['repayment_date'] == grouped.transform(lambda x: x.nsmallest(2).max())]['repayment_amount']
# df['p2'] = grouped.transform(lambda x: x.nsmallest(2).max())
# df

,origination_date,p2
20,2019-05-31,3332200.33
41,2019-06-30,3011884.91
62,2019-07-31,2953335.55
83,2019-08-31,4082016.00
104,2019-09-30,3930445.60
125,2019-10-31,4682354.31
146,2019-11-30,4637701.69
167,2019-12-31,4947764.21
188,2020-01-31,4696910.48
209,2020-02-29,6142911.08


In [251]:
df.groupby('origination_date')['repayment_amount'].apply(list).reset_index(name='repayments')

,origination_date,repayments
0,2019-05-31,"[1443069.08, 3332200.33, 1328138.75, 928085.74..."
1,2019-06-30,"[1392751.6, 3011884.91, 1237868.7, 970929.28, ..."
2,2019-07-31,"[1537650.24, 2953335.55, 1208316.08, 879375.19..."
3,2019-08-31,"[1617681.94, 4082016.0, 1387474.94, 1247623.59..."
4,2019-09-30,"[1992242.84, 3930445.6, 1394620.78, 1227905.58..."
5,2019-10-31,"[2289453.76, 4682354.31, 1659503.89, 1165897.0..."
6,2019-11-30,"[2162283.09, 4637701.69, 1576348.23, 1144559.9..."
7,2019-12-31,"[2402403.37, 4947764.21, 1523145.18, 1245452.3..."
8,2020-01-31,"[2502066.86, 4696910.48, 1505493.21, 1182983.9..."
9,2020-02-29,"[2833811.35, 6142911.08, 1833677.81, 1317065.7..."


In [264]:
df.groupby('origination_date')['repayment_amount'].apply(list).reset_index(name='repayments')['repayments']

# grouped = df.groupby('origination_date')['repayment_date']

## Get the second repayment (p2)

# df[df['repayment_date'] == grouped.transform(lambda x: x.nsmallest(2).max())][['origination_date', 'repayment_amount']].rename(columns={'repayment_amount':'p2'})


0     [1443069.08, 3332200.33, 1328138.75, 928085.74...
1     [1392751.6, 3011884.91, 1237868.7, 970929.28, ...
2     [1537650.24, 2953335.55, 1208316.08, 879375.19...
3     [1617681.94, 4082016.0, 1387474.94, 1247623.59...
4     [1992242.84, 3930445.6, 1394620.78, 1227905.58...
5     [2289453.76, 4682354.31, 1659503.89, 1165897.0...
6     [2162283.09, 4637701.69, 1576348.23, 1144559.9...
7     [2402403.37, 4947764.21, 1523145.18, 1245452.3...
8     [2502066.86, 4696910.48, 1505493.21, 1182983.9...
9     [2833811.35, 6142911.08, 1833677.81, 1317065.7...
10    [2843285.54, 6228477.78, 1585123.49, 1425436.3...
11    [3332800.29, 6476251.76, 1747437.15, 1419428.6...
12    [3575896.21, 7636995.58, 1542705.08, 1376501.7...
13    [3746372.58, 8983764.16, 1653631.68, 1512916.3...
14    [3470149.61, 8030091.09, 1612000.2, 1357290.46...
15    [3510328.95, 8374134.7, 1564208.19, 1362121.91...
16     [3976808.32, 7065477.24, 1385481.66, 1115017.68]
17                 [3667678.21, 8752706.73, 1503

In [229]:
# grouped = df.groupby('bq_market_id')['bq_back_price']
# df['second_lowest'] = grouped.transform(lambda x: x.nsmallest(2).max())



  # p = [p1, p2]



def expected_repayment():


  for i in np.arange(31):

    calc1 = sum(p)

    calc2 = 1 - calc1

    calc3 = (i - 1) / 30

    calc4 = 1 - calc2

    calc5 = 1 + (calc4 * calc3)

    calc6 = p2 * np.log(calc5)

    pi = max(calc6, 0)

    p.append(pi)
  
  return p
	

In [230]:
# p = [p1, p2]

# for i in np.arange(31):

# pi = max(aa, 0)

# aa = p2 * np.log(bb)

# bb = 1 + (cc*dd)

# cc = 1 - ee

# dd = (i - 1) / 30

# ee = 1 - ff

# ff = sum(p)
	

In [231]:
np.arange(31)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])